# Kaggle Titanic Pipeline 

Utility Classes

In [1]:
# Definition of the CategoricalEncoder class, copied from PR #9151.
# Just run this cell, or copy it to your code, do not try to understand it (yet).

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse
import numpy as np

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

        
#Data Frame Selector
from sklearn.base import BaseEstimator, TransformerMixin

# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

Data Preprocessing

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

DATA_FILE_PATH = 'data/train.csv'


# load data
def loadData(filepath=DATA_FILE_PATH):
    return pd.read_csv(filepath)

title_map = {'Sir' : ('Capt', 'Don', 'Major', 'Sir'), \
              'Lady' : ('Dona', 'Lady', 'the Countess', 'Jonkheer', 'Mme', 'Mlle') , 'Miss' : ('Miss', 'Ms')}  


def findTitle(name):
    title = name.split(",")[1].strip().split(".")[0]
    for uniqTitle, titles in title_map.items():
        if title in titles:
            return uniqTitle
    return title

# Rather using median values of age, the following snippet derives the value from parent/child and sibling/spouse data 
def ageImputer(data):
    parch = data["Parch"]; sibsp = data["SibSp"]
    if data["Age"] >= 0:
        return data["Age"]    
    if (parch == 0 and sibsp == 0):
        return 25
    elif (parch == 0 and sibsp == 1) or (parch >= 1 and sibsp >= 1):
        return 40
    elif parch >= 1 and parch < 3:
        return 9
    else:
        return 56
    
def preProcessAndGet(filepath):
    titanic_data = loadData(filepath)
    titanic_data["title"] = titanic_data["Name"].apply(findTitle)
    titanic_data["Family_Count"] = titanic_data["SibSp"] + titanic_data["Parch"]
    drop_columns = ["Cabin", "Name", "Ticket"]
    for col in drop_columns:
        titanic_data = titanic_data.drop(col, axis=1)
    
     
    titanic_data["Age"].fillna(-1, inplace=True)
    tit_age = titanic_data.apply(lambda x : ageImputer(x), axis=1)
    titanic_data["Age"] = tit_age
    titanic_data["Embarked"].fillna("S", inplace=True)
    
    # Label encoding male/female and embarkment
#     encoder = LabelEncoder()
#     embarked = titanic_data["Embarked"]
#     titanic_data["Embarked"] = encoder.fit_transform(embarked)
#     
#     encoder = LabelEncoder()
#     sex = titanic_data["Sex"]
#     titanic_data["Sex"] = encoder.fit_transform(sex)
    return titanic_data



Main Training/Testing Pipleline Code

In [3]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import FeatureUnion

DATA_FILE_PATH = 'data/train.csv'
TEST_FILE_PATH = 'data/test.csv'
titanic_data = preProcessAndGet(DATA_FILE_PATH)
titanic_label = titanic_data["Survived"]
titanic_data.drop("Survived", axis=1, inplace=True)
titanic_passenger_ids = titanic_data["PassengerId"]
titanic_data.drop("PassengerId", axis=1, inplace=True)

cat_attribs = ["Embarked", "Sex", "title"]
titanic_data_num = titanic_data.copy()
for cat_attrb in cat_attribs:
    titanic_data_num = titanic_data_num.drop(cat_attrb, axis=1)
num_attribs = titanic_data_num.columns.values.tolist()
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('cat_encoder', CategoricalEncoder(encoding="onehot-dense"))
    ])
    
full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])


Prediction

In [4]:
# titanic_data_preapred = full_pipeline.fit_transform(titanic_data)

# test_titanic_data = crt.preProcessAndGet(TEST_FILE_PATH)
# test_titanic_passenger_ids = test_titanic_data["PassengerId"]
# test_titanic_data.drop("PassengerId", axis=1, inplace=True)
# Run training program

# logReg = LinearRegression()
# logReg.fit(titanic_data_preapred, titanic_label)
# titanic_predictions = logReg.predict(titanic_data_preapred)
# log_mse = mean_squared_error(titanic_label, titanic_predictions)
# log_mse_rmse = np.sqrt(log_mse)
# print(log_mse)

test_titanic_data = preProcessAndGet(TEST_FILE_PATH)
test_titanic_passenger_ids = test_titanic_data["PassengerId"]
test_titanic_data.drop("PassengerId", axis=1, inplace=True)

full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("logRegression", LinearRegression())
    ])

full_pipeline_with_predictor.fit(titanic_data, titanic_label)
predict_labels = pd.Series(full_pipeline_with_predictor.predict(test_titanic_data)).apply(lambda x : 1 if x >= 0.5 else 0)

result = pd.DataFrame({'PassengerId': test_titanic_passenger_ids, 'Survived':predict_labels})
result.to_csv('sabya_submission.csv', index=False)

result

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


Grid Search on RandomForest and finding the best parameters

In [5]:
# scores = cross_val_score(full_pipeline_with_predictor, titanic_data, titanic_label,
#                          scoring="neg_mean_squared_error", cv=20)
# log_rmse_scores = np.sqrt(-scores)
#   
#    
# def display_scores(scores):
#     print("Scores:", scores)
#     print("Mean:", scores.mean())
#     print("Standard deviation:", scores.std())
#   
#    
# display_scores(log_rmse_scores)

from sklearn.model_selection import GridSearchCV
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]
forest_reg = RandomForestRegressor(random_state=42, n_estimators=30, max_features=4)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error')
grid_search.fit(full_pipeline.fit_transform(titanic_data), titanic_label)
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

Random Search on SVM and finding best parameters

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

# see https://docs.scipy.org/doc/scipy-0.19.0/reference/stats.html
# for `expon()` and `reciprocal()` documentation and more probability distribution functions.

# Note: gamma is ignored when kernel is "linear"
param_distribs = {
        'kernel': ['linear', 'rbf'],
        'C': reciprocal(20, 200000),
        'gamma': expon(scale=1.0),
    }

svm_reg = SVR()
rnd_search = RandomizedSearchCV(svm_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error',
                                verbose=2, n_jobs=2, random_state=42)
rnd_search.fit(full_pipeline.fit_transform(titanic_data), titanic_label)
rnd_search.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=629.782329591, gamma=3.01012143092, kernel=linear .............
[CV] C=629.782329591, gamma=3.01012143092, kernel=linear .............
[CV]  C=629.782329591, gamma=3.01012143092, kernel=linear, total= 1.4min
[CV] C=629.782329591, gamma=3.01012143092, kernel=linear .............
[CV]  C=629.782329591, gamma=3.01012143092, kernel=linear, total= 1.5min
[CV] C=629.782329591, gamma=3.01012143092, kernel=linear .............
[CV]  C=629.782329591, gamma=3.01012143092, kernel=linear, total= 1.3min
[CV] C=629.782329591, gamma=3.01012143092, kernel=linear .............
[CV]  C=629.782329591, gamma=3.01012143092, kernel=linear, total= 2.9min
[CV] C=26290.2064643, gamma=0.908446969632, kernel=rbf ...............
[CV]  C=26290.2064643, gamma=0.908446969632, kernel=rbf, total= 1.1min
[CV] C=26290.2064643, gamma=0.908446969632, kernel=rbf ...............
[CV]  C=629.782329591, gamma=3.01012143092, kernel=linear, total= 2.6min
[CV] C

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 12.0min


[CV]  C=21.3449536726, gamma=0.0233325235983, kernel=linear, total=   3.6s
[CV] C=5603.27031743, gamma=0.150234528727, kernel=rbf ...............
[CV]  C=21.3449536726, gamma=0.0233325235983, kernel=linear, total=   4.6s
[CV] C=5603.27031743, gamma=0.150234528727, kernel=rbf ...............
[CV]  C=5603.27031743, gamma=0.150234528727, kernel=rbf, total=  14.7s
[CV] C=5603.27031743, gamma=0.150234528727, kernel=rbf ...............
[CV]  C=5603.27031743, gamma=0.150234528727, kernel=rbf, total=  20.2s
[CV] C=5603.27031743, gamma=0.150234528727, kernel=rbf ...............
[CV]  C=5603.27031743, gamma=0.150234528727, kernel=rbf, total=  10.8s
[CV] C=5603.27031743, gamma=0.150234528727, kernel=rbf ...............
[CV]  C=5603.27031743, gamma=0.150234528727, kernel=rbf, total=  22.1s
[CV] C=157055.109894, gamma=0.26497040005, kernel=rbf ................
[CV]  C=5603.27031743, gamma=0.150234528727, kernel=rbf, total=  17.8s
[CV] C=157055.109894, gamma=0.26497040005, kernel=rbf ...............

[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed: 22.4min finished


{'C': 113564.03940586245, 'gamma': 0.00077906923665822954, 'kernel': 'rbf'}